# Installations



## 1. Google Colab

In [1]:
!pip install -q pyomo

### glpk

In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

### cbc

In [3]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 125129 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

### ipopt

In [4]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install -q pyomo
  !wget -N -q "https://github.com/Huy-Quang-Dao/SolverColab/raw/master/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64

In [5]:
!file ipopt-linux64.zip


ipopt-linux64.zip: Zip archive data, at least v2.0 to extract, compression method=deflate


### HiGHS

In [6]:
!pip install highspy

# A Step-by-Step Introduction for Pyomo Implementation

## 1. Linear Programming Formulation for Diet Problem

### Problem Statement

A dietitian is planning a meal that meets the daily nutritional requirements for calories, protein, and vitamins at a minimum cost.

| Food Item | Cost (\$) | Calories | Protein (g) | Vitamins (\% Daily) |
|-----------|-----------|----------|-------------|----------------------|
| Apple     | 1         | 100      | 0.5         | 2                    |
| Bread     | 0.50      | 200      | 4           | 0                    |
| Milk      | 2         | 150      | 8           | 10                   |
| Egg       | 0.30      | 70       | 6           | 0                    |

**Daily Nutritional Requirements:**
- Calories: 500
- Protein: 50g
- Vitamins: 100\%

Now we look forward to how a dietitian can bank on solving a LP model for the desired objective.




In [13]:
from pyomo.environ import *

# Model
model = ConcreteModel()

# Sets
model.foods = Set(initialize=['Apple', 'Bread', 'Milk', 'Egg'])
model.nutrients = Set(initialize=['Calories', 'Protein', 'Vitamins'])

# Data: Cost, nutritional content, and requirements
cost_data = {'Apple': 1, 'Bread': 0.5, 'Milk': 2, 'Egg': 0.3}
nutrients_data = {
    ('Apple', 'Calories'): 100, ('Apple', 'Protein'): 0.5, ('Apple', 'Vitamins'): 2,
    ('Bread', 'Calories'): 200, ('Bread', 'Protein'): 4, ('Bread', 'Vitamins'): 0,
    ('Milk', 'Calories'): 150, ('Milk', 'Protein'): 8, ('Milk', 'Vitamins'): 10,
    ('Egg', 'Calories'): 70, ('Egg', 'Protein'): 6, ('Egg', 'Vitamins'): 0
}

requirements_data = {'Calories': 500, 'Protein': 50, 'Vitamins': 100}


### Model

Define decision variables: $y_1$ for Apples, $y_2$ for Bread, $y_3$ for Milk, $y_4$ for Eggs. $y_i$ represents the quantity of each food item.

Minimize: $y_1 + 0.5y_2 + 2y_3 + 0.3y_4$

Subject to:
- $100y_1 + 200y_2 + 150y_3 + 70y_4 \geq 500$
- $0.5y_1 + 4y_2 + 8y_3 + 6y_4 \geq 50$
- $2y_1 + 0y_2 + 10y_3 + 0y_4 \geq 100$

Constraints: $y_1, y_2, y_3, y_4 \geq 0$

The three listed constraints help to ensure dietary requirements for calories, protein, and vitamins are met.

### Implementation


In [14]:
# Parameters
model.cost = Param(model.foods, initialize=cost_data)
model.content = Param(model.foods, model.nutrients,
                        initialize=nutrients_data)
model.requirements = Param(model.nutrients,
                            initialize=requirements_data)

# Decision Variables
model.x = Var(model.foods, domain=NonNegativeReals)

# Nutrient Constraints
def nutrient_rule(model, nutrient):
    return sum(model.content[f, nutrient] * model.x[f]
                for f in model.foods) >= model.requirements[nutrient]
model.NutrientConstraints = Constraint(model.nutrients, rule=nutrient_rule)

# Objective Function
model.total_cost = Objective(expr=
    sum(model.cost[f] * model.x[f] for f in model.foods),
    sense=minimize)

# Solve the model using a solver
# SolverFactory("glpk").solve(model)
# SolverFactory("cbc").solve(model)
# SolverFactory("ipopt").solve(model)
SolverFactory("appsi_highs").solve(model)

# Print the solution
for f in model.foods:
    print(f"{f}: {model.x[f].value}")

# Print the total cost
print('\n',f"Total Cost: {model.total_cost.expr()}")

Apple: 0.0
Bread: 0.0
Milk: 10.0
Egg: 0.0

 Total Cost: 20.0


## 2. Binary Knapsack Problem

### Problem Statement

Consider a hiker who needs to choose the most valuable items for a hike without overloading the backpack.

- Items:
  - Tent (Value: $120, Weight: 2kg)
  - Stove (Value: $80, Weight: 1kg)
  - Food (Value: $60, Weight: 1kg)

- Backpack capacity: 3.5kg

Objective: Maximize the value of items in the backpack.


In [23]:
from pyomo.environ import *

# Model
model = ConcreteModel()

# Sets
model.item = Set(initialize=['Tent', 'Stove', 'Food'])

# Data
items_value = {
    ('Tent'): 120,
    ('Stove'): 80,
    ('Food'): 60
}

items_weight = {
    ('Tent'): 2,
    ('Stove'): 1,
    ('Food'): 1
}

pack_capacity = 3.5


### Model

Define binary decision variables: $x_1$ for Tent, $x_2$ for Stove, $x_3$ for Food.
> $x_i = 1$ if the item is chosen, and 0 otherwise.

Maximize $120x_1 + 80x_2 + 60x_3$

Subject to $2x_1 + x_2 + x_3 \leq 3.5$

where $x_1, x_2, x_3 \in \{0, 1\}$.



### Implementation

In [24]:
# Parameters
model.items_value = Param(model.item,initialize=items_value)
model.items_weight = Param(model.item,initialize=items_weight)
model.pack_capacity = Param(initialize=pack_capacity)

# Decision Variables (Note: defined to be Binary)
model.x = Var(model.item, domain=Binary)

# Weight Constraint
model.WeightConstraint = Constraint(expr = sum(model.items_weight[i] * model.x[i]
                for i in model.item) <= model.pack_capacity)

# Objective Function
model.total_value = Objective(expr=
    sum(model.items_value[i] * model.x[i] for i in model.item),
    sense=maximize)

# Solve the model using a solver
SolverFactory("glpk").solve(model)
# SolverFactory("cbc").solve(model)
# SolverFactory("ipopt").solve(model)
# SolverFactory("appsi_highs").solve(model)

# Print the solution
for i in model.item:
    print(f"{i}: {model.x[i].value}")

# Print the total cost
print('\n',f"Total Value: {model.total_value.expr()}",'\n')

print(f"'1 means that the item is selected'")

Tent: 1.0
Stove: 1.0
Food: 0.0

 Total Value: 200.0 

'1 means that the item is selected'
